In [ ]:
# install & download necessary stuffs

!pip install --upgrade --no-cache-dir gdown
!pip install transformers
# !pip install transformers==2.8.0
!pip install datasets

!git clone https://github.com/jianguoz/Few-Shot-Intent-Detection.git
!cp -r /content/Few-Shot-Intent-Detection/Datasets/BANKING77 ./BANKING77


# !gdown --id 1n2zigNDiIArWtsiV9iUQLwfSBgtNn7ws
# !unzip fine-tune_checkpoint.zip

!gdown --id 1-4E0eEjyp7n_F75TEh7OKrpYPK4GLNoE
!unzip post-train_checkpoint.zip

!git clone https://github.com/hanjanghoon/BERT_FP.git


!cp -r post-train/ubuntu25 /content/BERT_FP/FPT/PT_checkpoint

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 50.4 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 6.6 MB 46.2 MB/s 
     |████████████████████████████████| 596 kB 38.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 5.1 MB/s 
     |████████████████████████████████| 136 kB 59.9 MB/s 
     |████████████████████████████████| 1.1 MB 52.2 MB/s 
     |████████████████████████████████| 212 kB 59.0 MB/s 
     |████████████████████████████████| 127 kB 56.5 MB/s 
     |████████████████████████████████| 144 kB 60.6 MB/s 
     |████████████████████████████████| 271 kB 61.7 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

# Setup

In [ ]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertConfig, BertModel, BertTokenizer, BertForSequenceClassification
from transformers import AdamW
import random

RANDOM_SEED = 42

torch.manual_seed(RANDOM_SEED)

In [ ]:
def load_examples(file_path, do_lower_case=True):
    examples = []
    
    with open('{}/seq.in'.format(file_path),'r',encoding="utf-8") as f_text, open('{}/label'.format(file_path),'r',encoding="utf-8") as f_label:
        for text, label in zip(f_text, f_label):
            
            e = Inputexample(text.strip(),label=label.strip())
            examples.append(e)
            
    return examples

class Inputexample(object):
    def __init__(self,text_a,label = None):
        self.text = text_a
        self.label = label

In [ ]:
N = 10
data = []
labels = []
train_samples = []
train_labels = []
embed_dim = 768
batch_size = 4
lr= 1e-5  
temp = 0.3  
lamda = 0.01  
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
path_5s = f'./BANKING77/train_5/'
path_10s = f'./BANKING77/train_10/'
path_test = f'./BANKING77/test/'
path_valid = f'./BANKING77/valid/'

In [ ]:
if N == 5:
    train_samples = load_examples(path_5s)
elif N == 10:
    train_samples = load_examples(path_10s)
print(f'n == {N}')
print((train_samples[55].text))
print((train_samples[55].label))

class CustomTextDataset(Dataset):
    def __init__(self,labels,text,batch_size,repeated_label:bool=False):
        self.labels = labels
        self.text = text
        self.batch_size = batch_size 
        self.count = 0 
        self.repeated_label = repeated_label
        # to use when training with supervise contrastive loss
        if self.repeated_label == True:
            self.used_idx = []
           # write the code here
        #    pass
          

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        
        label = self.labels[idx]
        
        data = self.text[idx]
        
        sample = {"Class": label,"Text": data}


    
        return sample



In [ ]:
# https://downgit.github.io/#/home?url=https:%2F%2Fgithub.com%2Fjianguoz%2FFew-Shot-Intent-Detection%2Ftree%2Fmain%2FDatasets%2FHWU64

# write code here : for downloading validation samples
valid_samples = load_examples(path_valid)
#write code here : for downloading test samples
test_samples = load_examples(path_test)
# preprocess for training 
for i in range(len(train_samples)):
    data.append(train_samples[i].text)
    labels.append(train_samples[i].label)
    
# write code here :preprocess validation samples
valid_data = []
valid_labels = []
for i in range(len(valid_samples)):
    valid_data.append(valid_samples[i].text)
    valid_labels.append(valid_samples[i].label)
# write code here : preprocess test samples
test_data1 = []
test_labels = []
for i in range(len(valid_samples)):
    test_data1.append(test_samples[i].text)
    test_labels.append(test_samples[i].label)
# dataloader for training 
train_data = CustomTextDataset(labels,data,batch_size=batch_size,repeated_label=True)
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)


# write code here : dataloader for validation
validation_data = CustomTextDataset(valid_labels,valid_data,batch_size=batch_size,repeated_label=True)
valid_loader = DataLoader(validation_data,batch_size=batch_size,shuffle=True)

# write code here : dataloader for test
test_data = CustomTextDataset(test_labels,test_data1,batch_size=batch_size,repeated_label=True)
test_loader = DataLoader(test_data,batch_size=batch_size,shuffle=True)

# got the number of unique classes from dataset
num_class = len(np.unique(np.array(labels)))

# get text label of uniqure classes
unique_label = np.unique(np.array(labels))

# map text label to index classes
label_maps = {unique_label[i]: i for i in range(len(unique_label))}

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import get_scheduler
from datasets import load_metric

from copy import deepcopy
from tqdm.auto import tqdm

def train_model(model, 
                train_loader, 
                num_epochs,
                optimizer,
                return_best_weight=True,
                valid_loader=None, 
                device=None):
    
    loaders = {"train_loader":train_loader, 
               "valid_loader":valid_loader} if valid_loader else {"train_loader":train_loader}
    
    num_training_steps = num_epochs * np.sum([len(loader) for loader in loaders.values()])
    progress_bar = tqdm(range(num_training_steps))

    lr_scheduler = get_scheduler(
        name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
    )

    metrics = {loader_type:load_metric('accuracy') for loader_type in loaders}
    metrics_history = {loader_type:[] for loader_type in loaders}
    
    best_valid_accuracy = 0
    best_epoch = None
    best_weight = None

    for epoch in range(num_epochs):
        print(f'\nEpoch: {epoch+1}', end=" ")
        for loader_type in loaders:

            model.train() if loader_type == 'train_loader' else model.eval()
            for batch in loaders[loader_type]:
                with torch.set_grad_enabled(loader_type == 'train_loader'):

                    sentence = batch["Text"]
                    inputs = tokenizer(sentence,padding=True,truncation=True,return_tensors="pt")
            
                    # move parameter to device
                    inputs = {k:v.to(device) for k,v in inputs.items()}

                    # map string labels to class idex
                    labels = [label_maps[stringtoId] for stringtoId in (batch['Class'])]

                    #print("show out: ",np.unique(labels, return_counts=True))
                    # convert list to tensor
                    labels = torch.tensor(labels).unsqueeze(0)
                    labels = labels.to(device)

                    outputs = model(**inputs,labels=labels)
                    loss = outputs.loss
                    if loader_type == 'train_loader':
                        loss.backward()

                        optimizer.step()
                        lr_scheduler.step()
                        optimizer.zero_grad()
                progress_bar.update(1)

                metrics[loader_type].add_batch(predictions=torch.argmax(outputs['logits'], dim=1), references=labels.view(-1))                
            
            current_accuracy = metrics[loader_type].compute()['accuracy']
            if loader_type == 'valid_loader' and current_accuracy > best_valid_accuracy:
                best_valid_accuracy = current_accuracy
                best_epoch = epoch
                best_weight = deepcopy(model.state_dict())
            metrics_history[loader_type].append(current_accuracy)

            print(f'{loader_type} accuracy: {current_accuracy:.5f}', end=" ")
    
    if return_best_weight and best_weight is not None:
        print(f'\nLoading best weight @ val_accuracy={best_valid_accuracy:.3f} @ epoch={best_epoch}...')
        model.load_state_dict(best_weight)

    return model, metrics_history

### BERT FP

In [ ]:
import sys
sys.path.append('/content/BERT_FP/Fine-Tuning')

class Args:
    """placeholder"""

%cd /content/BERT_FP

args = Args()

FT_model={
    'ubuntu': 'bert-base-uncased',
    'douban': 'bert-base-chinese',
    'e_commerce': 'bert-base-chinese'
}

args.task = 'ubuntu'
args.score_file_path = '/content'
args.do_lower_case = True

file_name="/content/BERT_FP/Fine-Tuning/BERT_finetuning.py"

replacing_texts = [
                   ('self.bert_model.bert.load_state_dict(state_dict=torch.load("./FPT/PT_checkpoint/ubuntu25/bert.pt"))', 'self.bert_model.bert.load_state_dict(state_dict=torch.load("./FPT/PT_checkpoint/ubuntu25/bert.pt"), strict=False)'),
                   ('self.bert_config = config_class.from_pretrained(FT_model[args.task],num_labels=1)', 'self.bert_config = config_class.from_pretrained(FT_model[args.task],num_labels=77)')
]

with open(file_name, "r+") as text_file:
    texts = text_file.read()
    for i in range(len(replacing_texts)):
        texts = texts.replace(replacing_texts[i][0], replacing_texts[i][1])
with open(file_name, "w") as text_file:
    text_file.write(texts)

from BERT_finetuning import NeuralNetwork

bert_fp = NeuralNetwork(args)
bert_fp_model = bert_fp.bert_model
bert_fp_tokenizer = bert_fp.bert_tokenizer 
%cd /content

# experiment

## Experiment 2

In [ ]:
basic_experiment_metrics_history = []


num_epochs = 50

NOT_HUGGINGFACE_PRETRAINED = {
    'bert_fp': {'model':bert_fp_model,
                'tokenizer':bert_fp_tokenizer}
}

MODEL_NAMES = ['bert-base-cased', 
               'roberta-base',
               'bert_fp',
               'johngiorgi/declutr-base',
               'princeton-nlp/sup-simcse-bert-base-uncased', 
               'princeton-nlp/sup-simcse-roberta-base',
               'cambridgeltl/trans-encoder-bi-simcse-bert-base',
               'cambridgeltl/trans-encoder-bi-simcse-roberta-base']

for model_name in MODEL_NAMES:
    print(model_name)
    if model_name in NOT_HUGGINGFACE_PRETRAINED:
        tokenizer = NOT_HUGGINGFACE_PRETRAINED[model_name]['tokenizer']
        model = NOT_HUGGINGFACE_PRETRAINED[model_name]['model']
    else:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=77)
    model = model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    trained_model, metrics_history = train_model(model=model, 
                                                train_loader=train_loader, 
                                                num_epochs=num_epochs,
                                                optimizer=optimizer,
                                                valid_loader=valid_loader, 
                                                device=device)
    
    basic_experiment_metrics_history.append(metrics_history)
    print('\n\n')

bert-base-cased


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/697 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at cambridgeltl/trans-encoder-bi-simcse-roberta-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cambridgeltl/trans-encoder-bi-simcse-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should pr

  0%|          | 0/28900 [00:00<?, ?it/s]


Epoch: 1 train_loader accuracy: 0.08052 valid_loader accuracy: 0.38961 
Epoch: 2 train_loader accuracy: 0.46234 valid_loader accuracy: 0.58896 
Epoch: 3 train_loader accuracy: 0.71688 valid_loader accuracy: 0.66818 
Epoch: 4 train_loader accuracy: 0.81299 valid_loader accuracy: 0.71558 
Epoch: 5 train_loader accuracy: 0.89481 valid_loader accuracy: 0.74675 
Epoch: 6 train_loader accuracy: 0.93506 valid_loader accuracy: 0.77792 
Epoch: 7 train_loader accuracy: 0.96494 valid_loader accuracy: 0.80065 
Epoch: 8 train_loader accuracy: 0.97143 valid_loader accuracy: 0.81494 
Epoch: 9 train_loader accuracy: 0.98312 valid_loader accuracy: 0.81558 
Epoch: 10 train_loader accuracy: 0.99221 valid_loader accuracy: 0.82273 
Epoch: 11 train_loader accuracy: 0.99351 valid_loader accuracy: 0.82662 
Epoch: 12 train_loader accuracy: 0.99740 valid_loader accuracy: 0.82857 
Epoch: 13 train_loader accuracy: 0.99740 valid_loader accuracy: 0.83506 
Epoch: 14 train_loader accuracy: 0.99740 valid_loader accur

## Experiment 3

In [ ]:
for index, (name, param) in enumerate(model.named_parameters()):                 
    print(index, name)

0 roberta.embeddings.word_embeddings.weight
1 roberta.embeddings.position_embeddings.weight
2 roberta.embeddings.token_type_embeddings.weight
3 roberta.embeddings.LayerNorm.weight
4 roberta.embeddings.LayerNorm.bias
5 roberta.encoder.layer.0.attention.self.query.weight
6 roberta.encoder.layer.0.attention.self.query.bias
7 roberta.encoder.layer.0.attention.self.key.weight
8 roberta.encoder.layer.0.attention.self.key.bias
9 roberta.encoder.layer.0.attention.self.value.weight
10 roberta.encoder.layer.0.attention.self.value.bias
11 roberta.encoder.layer.0.attention.output.dense.weight
12 roberta.encoder.layer.0.attention.output.dense.bias
13 roberta.encoder.layer.0.attention.output.LayerNorm.weight
14 roberta.encoder.layer.0.attention.output.LayerNorm.bias
15 roberta.encoder.layer.0.intermediate.dense.weight
16 roberta.encoder.layer.0.intermediate.dense.bias
17 roberta.encoder.layer.0.output.dense.weight
18 roberta.encoder.layer.0.output.dense.bias
19 roberta.encoder.layer.0.output.LayerNo

In [ ]:
# RoBERTa Architecture
LAST_FROZEN_LAYERS = [None, 4, 20, 36, 52, 68, 84, 100, 116, 132, 148, 164, 180, 196, 198]

In [ ]:
def freeze_layer(model, last_frozen_layer=None):
    # https://stackoverflow.com/a/65019579/13275449
    
    if last_frozen_layer is None:
        # No freezing 
        return 

    for index, (name, param) in enumerate(list(model.named_parameters())[:last_frozen_layer+1]):      
        param.requires_grad = False

    assert all([param.requires_grad == False for name, param in list(model.named_parameters())[:last_frozen_layer+1]]), f'Not all layers before layer {last_frozen_layer+1} is freezed'
    assert all([param.requires_grad == True for name, param in list(model.named_parameters())[last_frozen_layer+1:]]), f'Not all layers before after {last_frozen_layer+1} is trainable'

def calculate_trainable_parameters(model):
    return sum((p.numel() for p in model.parameters() if p.requires_grad))

In [ ]:
num_epochs = 50


metrics_history_all = []
time_taken_all = []

MODEL_NAMES = ['princeton-nlp/sup-simcse-roberta-base']
original_model = model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAMES[0], num_labels=77)
ORIGINAL_N_TRAINABLE_PARAMETERS = calculate_trainable_parameters(original_model)

for last_frozen_layer in LAST_FROZEN_LAYERS:
    print(str(last_frozen_layer))
    for model_name in MODEL_NAMES:
        
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=77)
        model = model.to(device)
        freeze_layer(model, last_frozen_layer=last_frozen_layer)
        print(f'last_frozen_layer is {last_frozen_layer}, ~{1-calculate_trainable_parameters(model) / ORIGINAL_N_TRAINABLE_PARAMETERS:.3f}% freezed')

        optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

        start = time.time()
        trained_model, metrics_history = train_model(model=model, 
                                                    train_loader=train_loader, 
                                                    num_epochs=num_epochs,
                                                    optimizer=optimizer,
                                                    valid_loader=valid_loader, 
                                                    device=device)
        end = time.time()
        time_taken = end - start
        time_taken_all.append(time_taken)
        metrics_history_all.append(metrics_history)
        print('\n\n')

Some weights of the model checkpoint at princeton-nlp/sup-simcse-roberta-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model 

None


Some weights of the model checkpoint at princeton-nlp/sup-simcse-roberta-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model 

last_frozen_layer is None, ~0.000% freezed


  0%|          | 0/28900 [00:00<?, ?it/s]

...


Epoch: 1 train_loader accuracy: 0.01558 valid_loader accuracy: 0.02727 
Epoch: 2 train_loader accuracy: 0.04286 valid_loader accuracy: 0.05649 
Epoch: 3 train_loader accuracy: 0.06753 valid_loader accuracy: 0.10260 
Epoch: 4 train_loader accuracy: 0.10519 valid_loader accuracy: 0.18117 
Epoch: 5 train_loader accuracy: 0.15195 valid_loader accuracy: 0.25714 
Epoch: 6 train_loader accuracy: 0.21429 valid_loader accuracy: 0.31299 
Epoch: 7 train_loader accuracy: 0.25584 valid_loader accuracy: 0.36818 
Epoch: 8 train_loader accuracy: 0.32208 valid_loader accuracy: 0.41364 
Epoch: 9 train_loader accuracy: 0.40519 valid_loader accuracy: 0.43442 
Epoch: 10 train_loader accuracy: 0.45065 valid_loader accuracy: 0.45974 
Epoch: 11 train_loader accuracy: 0.45974 valid_loader accuracy: 0.47792 
Epoch: 12 train_loader accuracy: 0.52078 valid_loader accuracy: 0.50260 
Epoch: 13 train_loader accuracy: 0.54026 valid_loader accuracy: 0.51299 
Epoch: 14 train_loader accuracy: 0.54286 valid_loader accur

Some weights of the model checkpoint at princeton-nlp/sup-simcse-roberta-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model 

last_frozen_layer is 198, ~1.000% freezed


  0%|          | 0/28900 [00:00<?, ?it/s]


Epoch: 1 train_loader accuracy: 0.01688 valid_loader accuracy: 0.02208 
Epoch: 2 train_loader accuracy: 0.01688 valid_loader accuracy: 0.02468 
Epoch: 3 train_loader accuracy: 0.02987 valid_loader accuracy: 0.02662 
Epoch: 4 train_loader accuracy: 0.02597 valid_loader accuracy: 0.03247 
Epoch: 5 train_loader accuracy: 0.02987 valid_loader accuracy: 0.03636 
Epoch: 6 train_loader accuracy: 0.03506 valid_loader accuracy: 0.03766 
Epoch: 7 train_loader accuracy: 0.03766 valid_loader accuracy: 0.03896 
Epoch: 8 train_loader accuracy: 0.04675 valid_loader accuracy: 0.04221 
Epoch: 9 train_loader accuracy: 0.04026 valid_loader accuracy: 0.05000 
Epoch: 10 train_loader accuracy: 0.04026 valid_loader accuracy: 0.05390 
Epoch: 11 train_loader accuracy: 0.05714 valid_loader accuracy: 0.06623 
Epoch: 12 train_loader accuracy: 0.05325 valid_loader accuracy: 0.07792 
Epoch: 13 train_loader accuracy: 0.07273 valid_loader accuracy: 0.08571 
Epoch: 14 train_loader accuracy: 0.09091 valid_loader accur